# Basketball challenge

In [ ]:
%load_ext autoreload
%autoreload 2

from PIL import Image
import torch
import torchvision
from Dataset import Basketball
from Dataprocess import Preprocess
from modules import FFNN, CNN
import random
from torch.utils.data import DataLoader

# Image transformation

In [ ]:
img_transformation = torchvision.transforms.Compose([
    torchvision.transforms.Resize((64,48)),
    torchvision.transforms.CenterCrop((48,48)),
    torchvision.transforms.ToTensor()
])


# Preprocessing Dataset

In [ ]:
dataprocess = Preprocess().background_subtractor

# Loading Dataset

In [ ]:
path = "data"
dataset = Basketball(path, split='training', num_frame = 100, img_transform = img_transformation, dataprocess=dataprocess)
trainset, testset = dataset.train_test_split()

In [ ]:
trainset = DataLoader(trainset, shuffle=True)
testset = DataLoader(testset, shuffle=True)

# Display Video

In [ ]:
#from IPython import display
#import torchvision.transforms.functional as F
#sets = DataLoader(dataset)
#for batch, target in sets:
#    print("here is the ",target)
#    view1 = batch[0][0]
#    for idx, img in enumerate(view1):
#        img1 = F.to_pil_image(img)
#        display.display(img1.resize((320,240)), Image.NEAREST)
#        display.clear_output(wait=True)
#    break

# CUDA

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

# Models

# Feed Forward Neural Network

In [ ]:
#print(dataset[0][0][0].shape)
variables = dataset[0][0][0].numel()
net = FFNN(variables, 2) # only 2 classifier hit or miss as output and variables input as parameters
#variables

# CNN

In [ ]:
in_channel = 1
out_class = 2
net = CNN() # only 2 classifier hit or miss as output and 1 channel as the shape of input will be (1,3,28x28)
net.to(device)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

from torchvision.transforms.functional import to_tensor, to_grayscale
from PIL import Image
im_gray = output[0]

im_gray = im_gray[None, ...]


plt.imshow(im_gray[0, 0], vmin=0., vmax=1.)


# Optimizer, Loss, Train, Test

In [ ]:
optimizer = torch.optim.SGD(net.parameters(),lr=0.001,momentum=0.4, nesterov=True)
loss_fn = torch.nn.CrossEntropyLoss().to(device)
net = net.to(device)

In [ ]:
def train(net, trainset):
    net.train()
    loss_sum = 0
    for batch, target in trainset:
        #batch is taken only as view1
        #change is later for both views
        inputs = batch[0][0].view(1,-1).to(device)
        #inputs = batch[0][0].unsqueeze(dim=0).to(device)
        target = torch.as_tensor(target).to(device)
        output = net(inputs)
        net.zero_grad()
        l = loss_fn(output, target)
        l.backward()
        optimizer.step()
        loss_sum += l.item()
        #print(loss_sum)
    print('\nTrain set: {}, Average loss: {:.4f}\n'.format(len(trainset), loss_sum/len(trainset)))

In [ ]:
 def test(net, testset):
    net.eval()
    correct = 0
    total = 0
    loss_sum = 0
    with torch.no_grad():
        for batch, target in testset:
            inputs = batch[0][0].view(1,-1).to(device)
            target = torch.as_tensor(target).to(device)
            output = net(inputs)
            #print(output)
            #print(torch.argmax(output))
            loss_sum += loss_fn(output,target) / len(testset)
            for _, prediction in enumerate(output):
                if torch.argmax(prediction) == target:
                    correct += 1
                total += 1
    print('\nTest set: {}, Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        len(testset), loss_sum, correct, len(testset), 100. * correct / len(testset)))


In [ ]:
epochs = 4
for epoch in range(0, epochs):
    train(net, trainset)
    test(net, testset)